In [1]:
import numpy as np

In [2]:
mass = [482.01, 108.14, 279.92, 80.91]  # mw of core, linker, ext, removed unit when a new bond is formed (HBr in this case)

def renorm(goal: float, mass: list) -> tuple:
    """retur.... Only works for situation like tri-core + dual-linker + dual-ext

    Args:
        goal (float): peak of interest on the mass spectrum.
        mass (list): [mw of core, mw of linker, mw of extender, mw of removed unit for each bond formation]

    Returns:
        tuple: c_nc, c_ne, c_nlf, c_goal.
    """
    m_c, m_l, m_e, m_rm = mass

    # a1 is the coefficient of number of c -> c_nc
    # a2 is the coefficient of number of e -> c_ne
    # a3 is the coefficient of number of l with one free termini (nl_f) -> c_nlf
    # a4 is the constant modified goal -> c_goal

    c_nc = m_c + m_e - 2 * m_rm
    c_ne = m_l + m_e - 2 * m_rm
    c_nlf = m_l - m_rm
    c_goal = goal + m_l - 2 * m_rm
    
    # a = [mass[0] + mass[1] - 2*mass[3], mass[2] + mass[1] - 2*mass[3], mass[1] - mass[3], goal + mass[1] - 2*mass[3]]
    
    return c_nc, c_ne, c_nlf, c_goal

def get_comb(a):
    N = [a[3]//a[0], a[3]//a[1], a[3]//a[2]]
    X = np.mgrid[:N[0]+1,:N[1]+1,:N[2]+1]
    return X

def unit_calc(X, a):
    x1, x2, x3 = X
    a1, a2, a3 = a[:-1]
    return a1*x1 + a2*x2 + a3*x3

def get_mass(res, mass):
    return res + 2*mass[3] - mass[1]   

def get_mask(X):
    x1, x2, x3 = X
    return x3 <= x1+2

def find_closest(res, a, k=5, mask=None):
    """return first k smallest err index and its results and its err (id, res, err)"""
    if mask is None:
        err = np.abs(res - a[-1])
    else:
        err = np.where(mask, np.abs(res - a[-1]), np.inf)
    idx = tuple(np.array(np.unravel_index(np.argsort(err, axis=None), err.shape))[:,:k])
    return idx, err[idx]

def get_fraction(idx):
    x1, x2, x3 = idx
    n1 = x1  # cores
    n3 = x2  # ext
    nf = x3  # free linker
    nb = n1 + n3 -1  # bonded linker
    n2 = nf + nb  # linkers
    n4 = nf + 2*nb  # HBr
    return n1, n2, n3, n4


In [3]:
def find_solution(goal, mass=mass, k=1):
    a = renorm(goal, mass)
    X = get_comb(a)
    summation = unit_calc(X, a)
    mask = get_mask(X)
    idx, err = find_closest(summation, a, k=k, mask=mask)    
    total_mass = get_mass(summation, mass)
    attempted_mass = np.sort(np.ravel(total_mass)[np.ravel(mask)])
    print("all attempted valid mass:")
    print(attempted_mass)
    return idx, total_mass[idx], total_mass[idx]-goal

In [4]:
idx, calc_mass, err = find_solution(563, k=3)

print("index:", np.array(idx).T)
print("calculated mass:", calc_mass)
print("error to goal:", err)

all attempted valid mass:
[  53.68   80.91  108.14  279.92  307.15  334.38  482.01  506.16  509.24
  533.39  536.47  560.62  563.7   708.25  735.48  762.71  789.94  934.49
  961.72  988.95 1016.18]
index: [[1 0 3]
 [0 2 2]
 [1 0 2]]
calculated mass: [563.7  560.62 536.47]
error to goal: [  0.7   -2.38 -26.53]


In [5]:
frac = np.array(get_fraction(idx)).T
print("[cores linkers exts rm_sites]:\n", frac)

[cores linkers exts rm_sites]:
 [[1 3 0 3]
 [0 3 2 4]
 [1 2 0 2]]
